## Determine new releases
Use requests and beautifulsoup4 to check and see which page on Ronin Flix site has the desired title, "Opera"


In [1]:
import requests
import bs4
import time

In [2]:
NR_URL = 'https://roninflix.com/collections/new-release'
CS_URL = 'https://roninflix.com/collections/coming-soon'


In [24]:
class Inquiry:
    
    def __init__(self, url):
        self.url = url
        self.page_response = self.pull_site()
        self.pull_time = time.ctime(time.time())
        self.titles = self.parse_titles()
    
    def __str__(self):
        ob_str = f'Queried {self.url} on {self.pull_time}. {str(len(self.titles))} titles found.'
        return ob_str
        
    def pull_site(self):
        site_response = requests.get(self.url, timeout=5)
        site_response.raise_for_status()
        return site_response
    
    def parse_titles(self):
        titles_list = []
        
        #raw page held in 'content' attribute of response obj
        soup = bs4.BeautifulSoup(self.page_response.content, 'html.parser')
        select_list = soup.select('.grid-view-item__title')
        
        for titles in select_list:
            titles_list.append(titles.getText())
        return titles_list

class InquiryRecord(Inquiry):
    
    def __init__(self, url, ptime, titles):
        self.url = url
        self.pull_time = ptime
        self.titles = titles
        

In [25]:
def check_movie(url):
    titles = Inquiry(url)
    
    for title in titles.titles:
        if 'Opera' in title:
            return(f'Opera found on Ronin {titles.url.split("/")[-1]} page')
    return(f'Opera is not yet on Ronin {titles.url.split("/")[-1]} page')



In [26]:
print(check_movie(NR_URL))
print(check_movie(CS_URL))

Opera is not yet on Ronin new-release page
Opera found on Ronin coming-soon page


## email notification
Send summary via email

In [27]:
def email_notify(msg_body):
    import os
    import smtplib

    sending_acct = os.environ.get('ALERTS_EMAIL')
    mail_pass = os.environ.get('ALERTS_PASSWORD')
    notify_email = os.environ.get('NOTIFY_EMAIL')

    with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
        #identify yourself
        smtp.ehlo()
        #encrypt connection
        smtp.starttls()
        smtp.ehlo()

        smtp.login(mail_acct, mail_pass)

        subject = 'Movie Notification'
        msg = f'Subject: {subject}\n\n{msg_body}'

        smtp.sendmail(sending_acct, notify_email, msg)

## save activity
to stop process when complete

### encoding and decoding custom types in json

In [28]:
#create fresh Inquiry
new_check = Inquiry(NR_URL)

#### encode

In [29]:
import json
#subclass jsonencoder
class InquiryEncoder(json.JSONEncoder):
    def default(self, inquiry):
        try:
            encode_dict = {
                '__inquiry__': True,
                'url': inquiry.url,
                'pull_time': inquiry.pull_time,
                'titles': inquiry.titles
                          }
            return encode_dict
        except:
            print('Could not properly encode')

In [30]:
with open('data_file.json', 'w') as write_file:
    json.dump(new_check, write_file, cls=InquiryEncoder)


#### decode

In [31]:
json_str = json.dumps(new_check, cls=InquiryEncoder)
json.loads(json_str)

{'__inquiry__': True,
 'pull_time': 'Fri Mar 22 18:18:39 2019',
 'titles': ['The Psychic',
  'Rituals',
  'Jungle Holocaust',
  'Tower of Evil (Horror on Snape Island)',
  'Curse III: Blood Sacrifice',
  'Robot Holocaust',
  'Bucktown',
  'Land of Doom',
  'Angel Unchained',
  'Mary, Mary, Bloody Mary',
  'Shakma'],
 'url': 'https://roninflix.com/collections/new-release'}

In [32]:
#make custom decode function
def decode_inquiry(dct):
    if '__inquiry__' in dct:
        return InquiryRecord(dct['url'], dct['pull_time'], dct['titles'])
    else:
        print('Can not rebuild InquiryRecord object from JSON')

In [33]:
with open('data_file.json') as inquiry_data:
    data = inquiry_data.read()
    recorded = json.loads(data, object_hook=decode_inquiry)

In [39]:
str(recorded)

'Queried https://roninflix.com/collections/new-release on Fri Mar 22 18:18:39 2019. 11 titles found.'

TypeError: object of type 'InquiryRecord' has no len()